In [16]:
import random

class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
        
    def getValue(self):
        return self.value
    
    def getCost(self):
        return self.calories
    
    def density(self):
        return self.getValue()/self.getCost()
    
    def __str__(self):
        return self.name + ": (" + str(self.value) + ", " + str(self.calories) + ")"

def buildMenu(items,maxVal,maxCost):
    menu = []
    for i in range(items):
        menu.append(Food(str(i), random.randint(1,maxVal), random.randint(1, maxCost)))
    return menu

def greedy(items, maxCost, keyFn):
    """
    Assumes items a list, maxCost >= 0
    keyFn maps elements of items to numbers
    """
    itemsCopy = sorted(items, key = keyFn, reverse = True)             # n log n
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):                                    # n
        if (totalCost + itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalValue += itemsCopy[i].getValue()
            totalCost += itemsCopy[i].getCost()
    return (result, totalValue)

def testGreedy(items, constraint, keyFn):
    taken, val = greedy(items, constraint, keyFn)
    print("Total value of items taken: " + str(val))
    for item in taken:
        print("  ", item)

def testAll(maxUnits):
    print("\nUse greedy by value to allocate", maxUnits, "calories")
    testGreedy(foods, maxUnits, Food.getValue)
    
    print("\nUse greedy by cost to allocate", maxUnits, "calories")
    testGreedy(foods, maxUnits, lambda x: 1/Food.getCost(x))
    
    print("\nUse greedy by density to allocate", maxUnits, "calories")
    testGreedy(foods, maxUnits, Food.density)
    
def maxVal(toConsider, avail):
    """
    Assumes toConsider a list of items
            avail a weight
    Returns a tuple of the total value of a solution to 0/1 knapsack problem
    and the items included in the solution.
    """
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getValue() > avail:
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        withVal, withTake = maxVal(toConsider[1:], avail - nextItem.getValue())
        withVal += nextItem.getValue()
        withoutVal, withoutTake = maxVal(toConsider[1:], avail)
        if withVal > withoutVal:
            result = (withVal, withTake + (nextItem,))
        else:
            result = (withoutVal, withoutTake)
    return result

def fastMaxVal(toConsider, avail):
    """
    Assumes toConsider a list of items
            avail a weight
    Returns a tuple of the total value of a solution to 0/1 knapsack problem
    and the items included in the solution.
    """
    def aux(toConsider, avail, memo):
        if (len(toConsider), avail) in memo:
            result = memo[(len(toConsider), avail)]
        elif toConsider == [] or avail == 0:
            result = (0, ())
        elif toConsider[0].getValue() > avail:
            result = aux(toConsider[1:], avail, memo)
        else:
            nextItem = toConsider[0]
            withVal, withTake = aux(toConsider[1:], avail - nextItem.getValue(), memo)
            withVal += nextItem.getValue()
            withoutVal, withoutTake = aux(toConsider[1:], avail, memo)
            if withVal > withoutVal:
                result = (withVal, withTake + (nextItem,))
            else:
                result = (withoutVal, withoutTake)
        memo[(len(toConsider), avail)] = result
        return result
    return aux(toConsider, avail, {})
    
def testMaxVal(foods, maxUnits, algorithm, printItems=True):
    print("Use search tree to allocate", maxUnits, "calories over", len(foods), "items.")
    val, taken = algorithm(foods, maxUnits)
    print("      Value", val, "taken, over", len(taken), "items taken.")
    if printItems:
        for item in taken:
            print("      -", item)
    

# buildMenu(10,100,100)

In [18]:
# Test maxVal
for items in range(5,25+1,5):
    menu = buildMenu(items, 90, 250)
    testMaxVal(menu, 750, maxVal, False)

Use search tree to allocate 750 calories over 5 items.
      Value 211 taken, over 5 items taken.
Use search tree to allocate 750 calories over 10 items.
      Value 439 taken, over 10 items taken.
Use search tree to allocate 750 calories over 15 items.
      Value 749 taken, over 13 items taken.
Use search tree to allocate 750 calories over 20 items.
      Value 750 taken, over 16 items taken.
Use search tree to allocate 750 calories over 25 items.
      Value 750 taken, over 13 items taken.


In [22]:
# Test fastMaxVal
for items in range(1200,1225+1,5):
    menu = buildMenu(items, 90, 250)
    testMaxVal(menu, 750, fastMaxVal, False)

Use search tree to allocate 750 calories over 1200 items.
      Value 750 taken, over 16 items taken.
Use search tree to allocate 750 calories over 1205 items.
      Value 750 taken, over 13 items taken.
Use search tree to allocate 750 calories over 1210 items.
      Value 750 taken, over 16 items taken.
Use search tree to allocate 750 calories over 1215 items.
      Value 750 taken, over 14 items taken.
Use search tree to allocate 750 calories over 1220 items.
      Value 750 taken, over 16 items taken.
Use search tree to allocate 750 calories over 1225 items.
      Value 750 taken, over 11 items taken.


In [21]:
import sys
sys.getrecursionlimit()
# sys.setrecursionlimit(4000)

2000